# **Financial Applicactions with ML & AI**

<img style="float: right;" src="https://github.com/torreblanca99/course_financial_applications/blob/develop/docs/img/logo_bourbaki.png?raw=1" width="100"/>

## **Module II:** Value Risk
#### Topic: Monte Carlo for Options Value

##### Name: Julio César Avila Torreblanca

- **Objective**: apply Monte Carlo to value options.

- **Contents**:
    - Notes:
        - Monte Carlo
    - Code:
        1. Libraries and parameters
        2. Read Data (Asset example)
        3. EDA
        4. Preparing Data
        5. Modeling
        6. Evaluation
----

# Class Notes: Markowitz's Portfolio Theory

## Introduction to Markowitz's Portfolio Theory

Harry Markowitz introduced the groundbreaking concept of portfolio selection in his 1952 paper, "Portfolio Selection," published in The Journal of Finance. He is considered the father of Modern Portfolio Theory (MPT), which mathematically formalizes the process of creating investment portfolios with a focus on optimizing returns for a given level of risk.

### Key Concepts

- **Mean-Variance Optimization**: The theory emphasizes the optimization of a portfolio's expected return for a specified level of risk, measured by the variance or standard deviation of returns.
- **Efficient Frontier**: A set of optimal portfolios that offer the highest expected return for a defined level of risk. Portfolios on the efficient frontier are considered optimal.
- **Diversification**: The strategy of spreading investments across various assets to reduce risk. Markowitz formalized diversification as a core principle of portfolio construction.

## Portfolio Construction

### Risk and Return

- The return of an asset $ i $ is defined as $ r_i = R_i - 1 $, where $ R_i $ is the rate of return of asset $ i $.
- The portfolio return $ r $ is given by:

$$
r = \sum_{i=1}^{n} r_i w_i
$$

where $ r_i $ is the return of asset $ i $ and $ w_i $ is the weight of asset $ i $ in the portfolio.

### Mean-Variance Optimization

The goal of Markowitz's mean-variance optimization is to choose the weights of the portfolio optimally to achieve the desired expected return with minimal volatility. The variance of the return is used as a proxy for risk:

- Let $ \mu_i = \mathbb{E}(r_i) $ denote the expected return of asset $ i $,
- Let $ m = (\mu_1, \mu_2, \ldots, \mu_n)^T $ represent the vector of expected returns,
- Let $ \Sigma $ denote the covariance matrix of asset returns,
- $ w = (w_1, w_2, \ldots, w_n)^T $ represents the weights of the portfolio.

The portfolio's expected return is $ m^T w $, and its variance is $ w^T \Sigma w $.

### Optimal Portfolio

For an acceptable level of expected return $ \mu_b $, an optimal portfolio is any portfolio that solves the following quadratic programming problem:

$$
\min_{w} \frac{1}{2} w^T \Sigma w
$$

subject to:

$$
m^T w = \mu_b
$$

$$
\sum_{i=1}^{n} w_i = 1
$$

### Conclusion

Markowitz's Portfolio Theory laid the groundwork for modern investment strategies by mathematically framing the trade-off between risk and return and underscoring the importance of diversification. The theory has inspired numerous further developments in finance, such as the Capital Asset Pricing Model (CAPM) and other portfolio optimization techniques.

---

# 1. Librerías y parámetros

In [1]:
import yfinance as yf

import pandas as pd
import numpy as np

import cvxopt as opt

import plotly.express as px

# 2. Rading data

In [ ]:
# Assets: Google, Apple, IBM, Microsoft, Netflix, Nvidia
data = yf.download(
        tickers = "GOOG AAPL IBM MSFT NFLX NVDA", # options
        period = "1y", # one year information
        interval = "1d", # daily information
    ).loc[:, 'Close']

data

# 3. EDA

In [ ]:
data.loc[:, [
    'AAPL', 
    'GOOG', 
    'IBM', 
    'MSFT', 
    'NFLX', 
    'NVDA']
         ].plot()

In [ ]:
data.columns

In [ ]:
data.loc[:, [
    'AAPL', 
    'GOOG', 
    'IBM', 
    'MSFT', 
    #'NFLX', 
    'NVDA']
         ].plot()

# 3. Preparing Data

## 3.1 Transformations

#### Anual Log- Returns

In [ ]:
annual_returns = np.log(data / data.shift()) * 252 # Para anualizar los retornos
annual_returns

### Expected values (Meas)

In [ ]:
# whole  year
annual_returns.iloc[0:250].mean()

In [ ]:
# first half of a year
annual_returns.iloc[0:125].mean()


## 3.2 Example of a manual portfolio

In [ ]:
# secong half of a year
mean_returns = annual_returns.iloc[125:].mean()
mean_returns

### Covanriances for second half of the year

In [ ]:
# covariance matrix
cov_returns = annual_returns.iloc[125:].cov()
cov_returns

### Set weights to each asset

In [ ]:
money = 1000 # amount of money available'''
omega = pd.Series([0.5, 0.1, 0.05, 0.15, 0.1, 0.1]) * money
omega
     

### Expected Return 

In [ ]:
np.dot(1+mean_returns, omega)

# 4. Modeling with Markovitz [finding optimal values for my portfolio]

https://pyportfoliopt.readthedocs.io/en/latest/UserGuide.html

If $ \mathbf{w}$ is the vector of weights of stocks with expected returns $ \mu $, then the portfolio return is equal to the weight of each stock multiplied by its return, that is, $ \mathbf{w}^T \mu $. The portfolio risk in terms of the covariance matrix $ \Sigma$ is given by 

$$
\sigma^2 = \mathbf{w}^T \Sigma \mathbf{w}.
$$

Which, a bit more "broken down," is written in the following 2 text boxes.

## Expected Portfolio Value Compared to Expected Value of Each Asset

I have 4 assets and a capital of $ v_0 $ pesos:

- Each asset has its expected return $ \mu_1, \mu_2, \mu_3, \mu_4 $.
- I invest in each asset $ w_1, w_2, w_3, w_4 $(such that $ w_1 + w_2 + w_3 + w_4 = v_0 $)

Then:

$$
E(R_p) = \sum_i w_i E(R_i) = \sum_i w_i \mu_i
$$

## Expected Portfolio Volatility Compared to Volatility of Each Asset

- Each asset has a volatility of $ \sigma_1, \sigma_2, \sigma_3, \sigma_4 $ respectively. Then the portfolio volatility is:

$$
\sigma_p^2 = \sum_i w_i^2 \sigma_i^2 + \sum_{j \neq i} w_i w_j \sigma_i\sigma_j\rho_{ij}
$$

where $ \sigma_i $ is the volatility of each asset, and $ \rho_{ij} $ is the correlation coefficient of the returns of asset $ i $ with asset $ j$ .

The Sharpe ratio is the excess return of the portfolio per unit of risk (volatility)

$$
SR = \frac{R_p - R_f}{\sigma}
$$

With this in mind, we will create an amount $ N $ of portfolios with random weights and will save the weights, return, volatility, and the Sharpe ratio for each one.

---


## 4.1 Simulation One Investment

In [ ]:
np.random.seed(1995)

N = 10000
N = 6  

k = annual_returns.shape[1]

weights = np.zeros((N, k))
returns = np.zeros(N)
volatilities = np.zeros(N)
sharpe_ratios = np.zeros(N)

weights2 = np.array([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])

weights.shape

In [ ]:
weights

In [ ]:
w = np.random.random(6)
w /= np.sum(w)
w

In [16]:
for i in range(N):
    w = np.random.random(k)
    w /= np.sum(w)

    w = weights2[i]  ##
    weights[i, :] = weights2[i]  ##

    weights[i, :] = w

    returns[i] = np.dot(mean_returns, w)

    volatilities[i] = np.sqrt(np.dot(w.T, np.dot(cov_returns, w)))         ###        w.T @ cov_returns @ w

    sharpe_ratios[i] = returns[i] / volatilities[i]

In [ ]:
weights

In [ ]:
# Pure Portafolios 
import matplotlib.pyplot as plt

px.scatter(x = volatilities, y = returns, color = sharpe_ratios,
           labels={
                     "x": "Volatilidad",
                     "y": "Retorno",
                     "color": "Razón de Sharpe"
                 }
           )
# plt.scatter(volatilities, returns, c = sharpe_ratios)

## 4.2 Simulation 10,000 samples

In [ ]:
np.random.seed(1995)

N = 10000 

k = annual_returns.shape[1]

weights = np.zeros((N, k))
returns = np.zeros(N)
volatilities = np.zeros(N)
sharpe_ratios = np.zeros(N)

weights2 = np.array([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])

weights.shape

In [ ]:
weights

In [ ]:
w = np.random.random(6)
w /= np.sum(w)
w

In [24]:
for i in range(N):
    w = np.random.random(k)
    w /= np.sum(w)

    weights[i, :] = w

    returns[i] = np.dot(mean_returns, w)

    volatilities[i] = np.sqrt(np.dot(w.T, np.dot(cov_returns, w)))         ###        w.T @ cov_returns @ w

    sharpe_ratios[i] = returns[i] / volatilities[i]

In [ ]:
weights

In [ ]:
# Pure Portafolios 
import matplotlib.pyplot as plt

px.scatter(x = volatilities, y = returns, color = sharpe_ratios,
           labels={
                     "x": "Volatilidad",
                     "y": "Retorno",
                     "color": "Razón de Sharpe"
                 }
           )
# plt.scatter(volatilities, returns, c = sharpe_ratios)

## 4.3 Find Optimal Portfolio

The optimization of the portfolio can be seen as a convex optimization problem, and a solution can be found using quadratic programming. If we denote the target return as $ \mu^*$ , the problem to be solved for the portfolio with only long positions is:

$$
\begin{align*}
\min_w \quad & \mathbf{w}^T \Sigma \mathbf{w} & (1)\\
\text{s.a.} \quad & \mathbf{w}^T \mu \geq \mu^* & (2)\\
& \mathbf{w}^T 1 = 1 & (3)\\
& w_i \geq 0 & (4)
\end{align*}
$$

To solve it, we will use the function `cvxopt.solvers.qp`. This requires that the optimization problem is in general form. Specifically, the general form of a quadratic programming problem is the following:

$$
\begin{align*}
\min_x \quad & \frac{1}{2} x^T P x + q^T x & (5)\\
\text{s.a.} \quad & Gx \preceq h & (6)\\
& Ax = b & (7)
\end{align*}
$$

In [ ]:
mu_star = 0.5 # minumun of return expected

G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

In [ ]:
results = opt.solvers.qp(P, q, G, h, A, b)
results

In [ ]:
np.set_printoptions(suppress=True) # not scientific notation

w = np.asarray(results['x']).reshape((-1))
w

In [ ]:
mean_returns

In [ ]:
# Validation
np.dot(mean_returns, w)

In [ ]:
# volatility
np.sqrt(results['primal objective'])

## 4.4 Optimal Values for multiple expected return values

In [ ]:
mu_stars = np.linspace(.0, .54, 51)
mu_stars

In [ ]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

mu_stars = np.linspace(.0, .6, 51)    #####################

ws = np.zeros((len(mu_stars), k))
mus = np.zeros(len(mu_stars))
sigmas = np.zeros(len(mu_stars))

for i, mu_star in enumerate(mu_stars):
    try:
        h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
        results = opt.solvers.qp(P, q, G, h, A, b)

        w = np.asarray(results['x']).reshape((-1))
        ws[i, :] = w
        mus[i] = np.dot(mean_returns, w)
        sigmas[i] = np.sqrt(results['primal objective'])
    except:
        print('domain error')

In [ ]:
import plotly.graph_objects as go

## Portfolios of an optimal frontier
fig = go.Figure()

fig.add_traces(
    [
        go.Scatter(
            x = volatilities, y = returns,
            marker = dict(
                color = sharpe_ratios,
                colorbar = dict(title="Razón de Sharpe")
            ),
            mode = 'markers',
            showlegend = False
        ),
        go.Scatter(
            x = sigmas, y = mus,
            mode = 'lines + markers',
            showlegend = False
        )
])

fig.update_layout(
    xaxis_title = 'Volatilidad',
    yaxis_title = 'Retorno'
)


## 4.5 Using another solver

#### Mean-Variance Choice

The optimal portfolio can also be obtained by maximizing, with respect to $ \mathbf{w} $,

$$
U(\mu, \Sigma; \mathbf{w}) = \mathbf{w}^T \mu - \frac{\delta}{2} \mathbf{w}^T \Sigma \mathbf{w}
$$

where $ \delta > 0 $ is the risk aversion parameter. The first-order condition to maximize it is

$$
\mu = \delta \Sigma \mathbf{w}
$$

which implies the following design for a portfolio with risk:

$$
\mathbf{w} = (\delta \Sigma)^{-1} \mu
$$

This is a system of linear equations that we can solve with `np.linalg.solve`.








In [ ]:
delta = .2

np.linalg.solve(delta * cov_returns,  mean_returns)
# it can give negative weights     

In [ ]:
data.columns

## Extra: https://python-advanced.quantecon.org/black_litterman.html